# Script

In [1]:
import numpy as np
import pandas as pd

## ICUp5NewWindow24

In [2]:
df_window = pd.read_csv('./data/output/ICUp5NewWindow24.csv')
df_window.head(3)

,subject_id,day,start_time,end_time,diff_end_start
0,12088486,1,2135-02-07 03:03:00,2135-02-08 03:03:00,24.0
1,12088486,2,2135-02-08 03:03:00,2135-02-09 03:03:00,24.0
2,12088486,3,2135-02-09 03:03:00,2135-02-10 03:03:00,24.0


In [3]:
df_window.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63005 entries, 0 to 63004
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   subject_id      63005 non-null  int64  
 1   day             63005 non-null  int64  
 2   start_time      63005 non-null  object 
 3   end_time        63005 non-null  object 
 4   diff_end_start  63005 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.4+ MB


In [4]:
df_window.describe().T

,count,mean,std,min,25%,50%,75%,max
subject_id,63005.0,1.506537e+07,2.875203e+06,10001884.0,12621244.0,15153936.0,17541976.0,19999840.0
day,63005.0,7.655091e+00,8.127624e+00,1.0,3.0,5.0,10.0,100.0
diff_end_start,63005.0,2.337211e+01,3.011440e+00,6.0,24.0,24.0,24.0,29.0


In [5]:
data_types = {
    'subject_id': 'int64',
    'day': 'int64',
    'start_time': 'datetime',
    'end_time': 'datetime'
}

In [6]:
correct_format = '%Y-%m-%d %H:%M:%S'

for col, dtype in data_types.items():
    if dtype == 'datetime':
        df_window[col] = pd.to_datetime(df_window[col], format=correct_format)
    else:
        df_window[col] = df_window[col].astype(dtype)

### A partir desse ponto iremos realizar os procedimentos referentes as extrações específicas

In [7]:
df_window.loc[:, 'evacuation'] = 0

## ICUp5Evacuacoes

In [8]:
df_evac = pd.read_csv('./data/input/ICUp5Evacuacoes.csv')
df_evac.head(3)

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,11051942,26447909,39663616,47220,2116-12-30 16:21:00.000000 UTC,2116-12-30 16:21:00.000000 UTC,224011,Hard,NaN,NaN,0
1,15331346,21138971,31923713,49804,2115-08-27 10:11:00.000000 UTC,2115-08-27 10:11:00.000000 UTC,224011,Hard,NaN,NaN,0
2,16010357,27424541,36391937,96611,2112-01-26 08:13:00.000000 UTC,2112-01-26 08:13:00.000000 UTC,224011,Hard,NaN,NaN,0


In [9]:
df_evac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65461 entries, 0 to 65460
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subject_id    65461 non-null  int64  
 1   hadm_id       65461 non-null  int64  
 2   stay_id       65461 non-null  int64  
 3   caregiver_id  65461 non-null  int64  
 4   charttime     65461 non-null  object 
 5   storetime     65461 non-null  object 
 6   itemid        65461 non-null  int64  
 7   value         65461 non-null  object 
 8   valuenum      0 non-null      float64
 9   valueuom      0 non-null      float64
 10  warning       65461 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 5.5+ MB


In [10]:
data_types = {
    'subject_id': 'int64',
    'charttime': 'datetime',
    'storetime': 'datetime',
    'value': 'category'
}

In [11]:
correct_format = '%Y-%m-%d %H:%M:%S.%f UTC'

for col, dtype in data_types.items():
    if dtype == 'datetime':
        df_evac[col] = pd.to_datetime(df_evac[col], format=correct_format)
    else:
        df_evac[col] = df_evac[col].astype(dtype)
        
df_evac.head(1)

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,11051942,26447909,39663616,47220,2116-12-30 16:21:00,2116-12-30 16:21:00,224011,Hard,NaN,NaN,0


In [12]:
for subject_id in df_evac.subject_id.unique():
    if df_window[df_window.subject_id == subject_id].shape[0]:
        charttime = df_evac[df_evac.subject_id == subject_id].charttime.to_list()
        
        df_temp = df_window[df_window.subject_id == subject_id]
        
        for item in charttime:
            for index, row in df_temp.iterrows():
                if row.start_time <= item <= row.end_time:
                    df_window.loc[index, 'evacuation'] += 1

## ICUpatients5

In [13]:
df_patients = pd.read_csv('./data/input/ICUpatients5.csv')
df_patients.head(3)

,subject_id,hadm_id,stay_id,gender,anchor_age,admittime,intime,outtime,deathtime,intervaloAdmIntUTI,tempoUTI,race,pesoadm,unidpesoadm,altura,unidaltura,imc
0,12088486,29875103,39348271,M,18,2135-02-07 02:57:00 UTC,2135-02-07 03:03:00 UTC,2135-02-12 00:25:00 UTC,2135-02-11 00:01:00 UTC,0,117,UNABLE TO OBTAIN,71.8,kg,178.0,cm,-99.9
1,19850244,27972658,32398411,F,18,2162-06-04 17:38:00 UTC,2162-06-05 15:25:42 UTC,2162-06-09 20:22:03 UTC,NaN,21,100,OTHER,57.7,kg,163.0,cm,-99.9
2,10562205,20149612,34317198,M,18,2149-04-29 23:19:00 UTC,2149-04-30 00:58:00 UTC,2149-05-09 20:54:46 UTC,NaN,1,235,WHITE,98.0,kg,180.0,cm,-99.9


In [14]:
data_types = {
    'subject_id': 'int64',
    'intime': 'datetime',
    'outtime': 'datetime',
}

In [15]:
correct_format = '%Y-%m-%d %H:%M:%S UTC'

for col, dtype in data_types.items():
    if dtype == 'datetime':
        df_patients[col] = pd.to_datetime(df_patients[col], format=correct_format)
    else:
        df_patients[col] = df_patients[col].astype(dtype)
        
df_patients = df_patients[data_types.keys()]

In [16]:
df_patients.loc[:, 'evacuation'] = 0
df_patients.loc[:, 'constipation'] = 0
df_patients.loc[:, 'diarrhea'] = 0

In [17]:
def get_evacuation(row: pd.Series) -> int:
    subject_id = row.subject_id
    
    return df_window[df_window.subject_id == subject_id].evacuation.sum()

def get_diarrhea(row: pd.Series) -> int:
    '''
        1 - se apresentar 4 ou mais dejeções numa janela de 24h;
        2 - se apresentar 4 ou mais dejeções numa janela de 24h por mais de 3 dias seguidos;
        3 - se repete o anterior (opção 2) ao longo do internamento;
        0 - se não apresenta nenhum dos comportamentos citados acima.
    '''
    DAYS = 3
    TARGET = 4
    
    subject_id = row.subject_id
    index = df_window[df_window.subject_id == subject_id].index
    
    control = 0
    for i in index:
        window = df_window.loc[i:i+DAYS - 1].evacuation
        
        if window.shape[0] != DAYS:
            continue
        
        if window.sum() >= TARGET * DAYS:
            control += 1
                
        if control >= 3:
            return 3
            
    if control:
        return 2
    
    if window.max() >= TARGET:
        return 1
        
    return 0

def get_constipation(row: pd.Series) -> int:
    '''
        1 - se não tem anotação de evacuações por mais de 5 janelas de 24h seguidas a partir do intime OU de alguma anotação de evacuação;
        2 - se apresenta o comportamento do item anterior e isso se repete ao longo do internamento;
        0 - se não apresenta nenhum dos comportamentos acima.
    '''
    DAYS = 5
    TARGET = 0
    
    subject_id = row.subject_id
    index = df_window[df_window.subject_id == subject_id].index
    
    control = 0
    for i in index:
        window = df_window.loc[i:i+DAYS - 1].evacuation
        
        if window.shape[0] != DAYS:
            continue
            
        if window.sum() == TARGET:
            control += 1
    
        if control >= 3:
            return 2
    
    if control:
        return 1
    
    return 0

In [18]:
df_patients.loc[:, 'evacuation'] = df_patients.apply(get_evacuation, axis=1)

In [19]:
df_patients.loc[:, 'diarrhea'] = df_patients.apply(get_diarrhea, axis=1)

In [20]:
df_patients.loc[:, 'constipation'] = df_patients.apply(get_constipation, axis=1)

In [21]:
df_patients.sort_values('evacuation', ascending=False).head(3)

,subject_id,intime,outtime,evacuation,constipation,diarrhea
851,11129835,2154-05-18 13:11:47,2154-07-18 13:42:25,235,2,3
3758,10699336,2158-01-07 19:25:07,2158-04-17 10:44:29,203,2,3
222,15479539,2138-05-25 01:01:36,2138-07-30 15:02:54,198,0,3


In [22]:
df_patients.sort_values('diarrhea', ascending=False).head(3)

,subject_id,intime,outtime,evacuation,constipation,diarrhea
2541,13545680,2141-11-30 20:58:00,2141-12-21 16:43:31,57,0,3
2165,14539412,2130-08-09 09:55:04,2130-10-01 20:38:17,186,1,3
2232,15219352,2146-04-11 09:12:15,2146-04-19 22:17:57,25,0,3


In [23]:
df_patients.sort_values('constipation', ascending=False).head(3)

,subject_id,intime,outtime,evacuation,constipation,diarrhea
3482,10791581,2111-10-09 10:17:50,2111-10-13 17:59:39,0,2,0
1778,15951517,2189-03-17 21:09:00,2189-04-04 22:07:14,5,2,0
1780,15272000,2164-05-03 16:30:47,2164-05-06 19:47:40,0,2,0


## Finalizando

In [24]:
df_final = df_patients.reset_index()[['subject_id', 'intime', 'outtime', 'evacuation', 'constipation', 'diarrhea']]
df_final.head()

,subject_id,intime,outtime,evacuation,constipation,diarrhea
0,12088486,2135-02-07 03:03:00,2135-02-12 00:25:00,12,0,2
1,19850244,2162-06-05 15:25:42,2162-06-09 20:22:03,20,0,2
2,10562205,2149-04-30 00:58:00,2149-05-09 20:54:46,0,2,0
3,17840418,2129-01-14 08:00:00,2129-01-17 16:52:18,3,0,1
4,17590005,2134-03-03 03:47:56,2134-03-06 16:58:23,14,0,2


In [25]:
df_final.to_csv('./data/output/ICUp5Evacuation.csv', index=False)